In [1]:
import routeur

LAMBDA_REQUETE = 0.5

C = 3

T_MAX = 1000

routeur.simul_fifo(LAMBDA_REQUETE, C, T_MAX)

[[0, 1],
 [0, 2],
 [0.6666666666666666, 1],
 [1.3333333333333333, 0],
 [2.6538056002423636, 0],
 [5.812367181373902, 1],
 [6.479033848040569, 0],
 [6.607169460617759, 1],
 [6.997238420254478, 1],
 [7.273836127284426, 0],
 [7.360209678454379, 0],
 [8.073742216112782, 0],
 [9.294537444222335, 1],
 [9.601207214070639, 2],
 [9.961204110889001, 1],
 [10.267873880737305, 0],
 [10.36583246519723, 0],
 [10.627870777555668, -1],
 [10.982548974531888, -1],
 [11.50057972242891, -1],
 [13.317759504005867, 0],
 [13.826834720874412, 1],
 [13.971681186292896, 2],
 [13.984426170672533, 1],
 [14.493501387541079, 0],
 [14.638347852959562, -1],
 [14.651092837339199, -2],
 [14.695935647706637, -2],
 [14.722873710326402, -2],
 [16.169860490902906, -2],
 [16.3284507924012, -1],
 [16.995117459067867, -2],
 [17.07787437319207, -1],
 [17.74454103985874, -2],
 [17.775945758791018, -2],
 [18.209954054014112, -2],
 [20.59447030169328, -1],
 [21.26113696835995, -2],
 [22.528131869879946, -1],
 [23.194798536546614,